In [101]:
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Turbo256
from bokeh.plotting import figure
from bokeh.layouts import Column, Row
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from pyproj import Transformer, CRS
  
P0_SANTILLANA = (40.706421, -3.87)
P0_WASHINGTON = (47.57, -122.27)
crs = CRS.from_epsg(3857)
transformer = Transformer.from_crs(crs.geodetic_crs, crs)
(x_min, y_min) = transformer.transform(*P0_WASHINGTON)
(x_max, y_max) = (x_min + 1000, y_min + 5000)

output_notebook()

def get_data_sources_for(usv_name, data):
  sensor = data.Source.eq(f'{usv_name}.sim_sensor')
  imu = data.Source.eq(f'{usv_name}.IMU')
  usv = pd.DataFrame()
  usv['crs'] = data[imu].apply(lambda x: transformer.transform(x['lat'], x['lon']), axis=1)
  usv['x'] = usv.apply(lambda x: x['crs'][0], axis=1)
  usv['y'] = usv.apply(lambda x: x['crs'][1], axis=1)
  measurements = pd.DataFrame()
  measurements['crs'] = data[sensor].apply(lambda x: transformer.transform(x['lat'], x['lon']), axis=1)
  measurements['x'] = measurements.apply(lambda x: x['crs'][0], axis=1)
  measurements['y'] = measurements.apply(lambda x: x['crs'][1], axis=1)
  measurements['WQ_O'] = data[sensor]['WQ_O']
  max = measurements['WQ_O'].max()
  max = max if max > 0 else 1
  measurements['color'] = 255*measurements['WQ_O'] / max
  measurements['color'] = measurements['color'].astype(int).apply(lambda x: Turbo256[x])
  return (
    ColumnDataSource(usv),
    ColumnDataSource(measurements)
  )

def plot(usv: list[str], usv_colors):
  fig = figure(
    x_axis_type="mercator",
    y_axis_type="mercator",
    plot_width=400,
    plot_height=400,
    x_range=(x_min, x_max), #(-122.35,-122.20),
    y_range=(y_min, y_max), #(47.5,47.8),
    title='USV Position & Measures',
    x_axis_label='Longitude',
    y_axis_label='Latitude'
  )
  fig.add_tile(get_provider(CARTODBPOSITRON))
  sources = [
    get_data_sources_for(name, data) for name in usv
  ]
  for (usv, sensor), color in zip(sources, usv_colors):
    fig.step('x', 'y', source=usv, line_width=2, color=color)
    fig.circle('x', 'y', source=sensor, line_width=2, size=5, color='color')
  return fig

Loading BokehJS ...

In [102]:
data = pd.read_excel('../data/USVData.xlsx')
fig = plot(('Red Leader',), usv_colors=('red',))
show(fig)

[(ColumnDataSource(id='22611', ...), ColumnDataSource(id='22612', ...))]


In [103]:
data = pd.read_excel('../data/TwoUSVData.xlsx')  #Lectura de resultados excel
fig = plot(('Red Leader', 'Red Two'), usv_colors=('red', 'blue'))
show(fig)


[(ColumnDataSource(id='23399', ...), ColumnDataSource(id='23400', ...)), (ColumnDataSource(id='23401', ...), ColumnDataSource(id='23402', ...))]
